# PPCA for Missing Data Imputation

Objectives:
1. Create synthetic low-rank Gaussian data and randomly remove 10% entries.
2. Run Probabilistic PCA (EM) that natively handles missing values to jointly learn parameters and infer latent variables.
3. Impute missing entries (posterior mean) and compare against ground-truth values using RMSE / MAE.

We also compare with a simple baseline (mean imputation) to show PPCA advantage.


In [1]:
import numpy as np
import math
from numpy.linalg import inv
import matplotlib.pyplot as plt

np.random.seed(42)

# I. Create synthetic data
n = 1000   # samples
d = 20     # observed dimension
k_true = 3 # latent dimension

# True parameters
W_true = np.random.randn(d, k_true)
# Make W well-conditioned
U, _, Vt = np.linalg.svd(W_true, full_matrices=False)
W_true = U @ np.diag(np.linspace(2.0,0.5,k_true)) @ Vt
mu_true = np.random.randn(d) * 0.5
sigma2_true = 0.1

Z_true = np.random.randn(n, k_true)
X_full = Z_true @ W_true.T + mu_true + np.random.randn(n, d) * math.sqrt(sigma2_true)

# Introduce 10% missing at random
missing_rate = 0.10
mask = np.random.rand(n, d) > missing_rate  # True = observed
X_obs = X_full.copy()
X_obs[~mask] = np.nan

print(f"Generated data: X_full shape {X_full.shape}, missing entries {(~mask).sum()} ({(~mask).mean()*100:.1f}%)")


Generated data: X_full shape (1000, 20), missing entries 1988 (9.9%)


In [2]:
def ppca_em_missing(X, k, max_iter=200, tol=1e-5, verbose=False, seed=0):
    """PPCA EM handling missing values (NaNs) via per-sample observed mask.
    X: (n,d) with NaNs for missing
    Returns model dict and posterior means for latent variables.
    """
    rng = np.random.default_rng(seed)
    X = np.asarray(X)
    n, d = X.shape
    mask_obs = ~np.isnan(X)  # boolean
    # Initialize: fill NaNs with column means for a quick start
    col_means = np.nanmean(X, axis=0)
    X_filled = np.where(mask_obs, X, col_means)
    mu = np.nanmean(X, axis=0)
    # init W random, isotropic noise
    W = rng.normal(scale=0.1, size=(d, k))
    sigma2 = 1.0
    I_k = np.eye(k)

    ll_prev = -np.inf
    for it in range(max_iter):
        # E-step: compute Ez_i and Ezz_i per sample using only observed dims
        Ez = np.zeros((n, k))
        Ezz = np.zeros((n, k, k))
        ll = 0.0
        for i in range(n):
            obs = mask_obs[i]
            x_i_obs = X[i, obs]
            W_obs = W[obs]
            mu_obs = mu[obs]
            M_i = W_obs.T @ W_obs + sigma2 * I_k
            M_i_inv = inv(M_i)
            Ez_i = (x_i_obs - mu_obs) @ W_obs @ M_i_inv
            Ez[i] = Ez_i
            Ezz[i] = sigma2 * M_i_inv + np.outer(Ez_i, Ez_i)
            # Accumulate approximate log-likelihood (only constants across iters matter for convergence check)
            # Not strictly needed; we'll use parameter diff for stopping.
        # M-step: update mu using observed residuals with current latent expectations
        # Impute expected X for missing dims to update mu and W.
        X_recon = np.zeros_like(X_filled)
        for i in range(n):
            X_recon[i] = mu + Ez[i] @ W.T
        # Use observed entries where present, else reconstructed for purpose of mean centering
        X_completed = np.where(mask_obs, X, X_recon)
        mu = X_completed.mean(axis=0)
        Xc = X_completed - mu
        # Update W using summed cross-covariances restricted to observed entries per sample
        # Accumulate Sx = sum_i xci_obs Ez_i^T and sum_Ezz
        Sx = np.zeros((d, k))
        sum_Ezz = np.zeros((k, k))
        for i in range(n):
            obs = mask_obs[i]
            xci_obs = Xc[i, obs]
            Sx[obs] += np.outer(xci_obs, Ez[i])
            sum_Ezz += Ezz[i]
        W_new = Sx @ inv(sum_Ezz)
        # Update sigma^2 using observed residuals only
        res_sum = 0.0
        obs_count = 0
        for i in range(n):
            obs = mask_obs[i]
            xci_obs = Xc[i, obs]
            recon_obs = W_new[obs] @ Ez[i]
            diff = xci_obs - recon_obs
            res_sum += diff @ diff
            # Add trace term from latent covariance
            res_sum += np.trace(W_new[obs] @ (Ezz[i] - np.outer(Ez[i], Ez[i])) @ W_new[obs].T)
            obs_count += obs.sum()
        sigma2_new = res_sum / obs_count
        # Convergence check
        param_change = np.linalg.norm(W_new - W) / (np.linalg.norm(W) + 1e-12) + abs(sigma2_new - sigma2)/(sigma2 + 1e-12)
        W = W_new
        sigma2 = max(sigma2_new, 1e-8)
        if verbose and it % 10 == 0:
            print(f"Iter {it}: sigma2={sigma2:.4f} change={param_change:.2e}")
        if param_change < tol:
            break
    return {'mu': mu, 'W': W, 'sigma2': sigma2, 'Ez': Ez, 'Ezz': Ezz}

model = ppca_em_missing(X_obs, k=3, max_iter=300, tol=1e-5, verbose=True)
print("Learned sigma2:", model['sigma2'])


Iter 0: sigma2=0.3736 change=1.55e+00
Iter 10: sigma2=0.1063 change=2.61e-02
Iter 20: sigma2=0.1058 change=2.92e-03
Iter 30: sigma2=0.1058 change=2.85e-03
Iter 40: sigma2=0.1058 change=2.84e-03
Iter 50: sigma2=0.1058 change=2.84e-03
Iter 60: sigma2=0.1058 change=2.84e-03
Iter 70: sigma2=0.1058 change=2.84e-03
Iter 80: sigma2=0.1058 change=2.84e-03
Iter 90: sigma2=0.1058 change=2.84e-03
Iter 100: sigma2=0.1058 change=2.84e-03
Iter 110: sigma2=0.1058 change=2.84e-03
Iter 120: sigma2=0.1058 change=2.84e-03
Iter 130: sigma2=0.1058 change=2.84e-03
Iter 140: sigma2=0.1058 change=2.84e-03
Iter 150: sigma2=0.1058 change=2.84e-03
Iter 160: sigma2=0.1058 change=2.84e-03
Iter 170: sigma2=0.1058 change=2.84e-03
Iter 180: sigma2=0.1058 change=2.84e-03
Iter 190: sigma2=0.1058 change=2.84e-03
Iter 200: sigma2=0.1058 change=2.84e-03
Iter 210: sigma2=0.1058 change=2.84e-03
Iter 220: sigma2=0.1058 change=2.84e-03
Iter 230: sigma2=0.1058 change=2.84e-03
Iter 240: sigma2=0.1058 change=2.84e-03
Iter 250: s

In [3]:
# II / III. Impute missing values and compare to truth

mu, W, Ez = model['mu'], model['W'], model['Ez']
X_ppca_recon = Ez @ W.T + mu  # posterior mean of complete data (ignoring noise)

# Extract only originally missing positions
missing_idx = np.where(~mask)  # tuple (rows, cols)
true_missing_vals = X_full[missing_idx]
imputed_ppca = X_ppca_recon[missing_idx]

# Baseline: column mean imputation (using observed-only means)
col_mean_obs = np.nanmean(X_obs, axis=0)
imputed_mean = col_mean_obs[missing_idx[1]]

rmse_ppca = np.sqrt(np.mean((imputed_ppca - true_missing_vals)**2))
mae_ppca = np.mean(np.abs(imputed_ppca - true_missing_vals))
rmse_mean = np.sqrt(np.mean((imputed_mean - true_missing_vals)**2))
mae_mean = np.mean(np.abs(imputed_mean - true_missing_vals))

print(f"Missing entries: {true_missing_vals.size}")
print(f"PPCA   RMSE: {rmse_ppca:.4f}  MAE: {mae_ppca:.4f}")
print(f"Mean   RMSE: {rmse_mean:.4f}  MAE: {mae_mean:.4f}")

# Optional: visualize a few feature distributions (true vs imputed)
import pandas as pd
sel_features = np.random.choice(d, 3, replace=False)
rows = []
for f in sel_features:
    idx_f = (missing_idx[1] == f)
    rows.append({
        'feature': f,
        'ppca_rmse': np.sqrt(np.mean((imputed_ppca[idx_f]-true_missing_vals[idx_f])**2)),
        'mean_rmse': np.sqrt(np.mean((imputed_mean[idx_f]-true_missing_vals[idx_f])**2))
    })
print(pd.DataFrame(rows))


Missing entries: 1988
PPCA   RMSE: 0.3560  MAE: 0.2821
Mean   RMSE: 0.6159  MAE: 0.4665
   feature  ppca_rmse  mean_rmse
0        9   0.302978   0.421618
1       18   0.378228   0.593970
2       11   0.382183   0.504472
